**Mahane Rag** - Hebrew Jewish book QA  
![Mahane Rag logo](logo.png)


in this project i attemped to make a halacha-chatbot.  
for that, i used "Yalkut Yosef" books wich cover all Halacha topics and were written in the current century

frist - import all the necessary libreries

In [ ]:
from transformers import AutoTokenizer, pipeline
import random
import torch
from transformers import BertModel
from pymongo import MongoClient
import pickle
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import streamlit as st
import os
import faiss
import numpy as np
import cohere
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.load.dump import dumps
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
import json
import os
import sys
import boto3
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import anthropic
import ssl


make a random seed

In [ ]:

random_seed = 42
random.seed(random_seed)

# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

**text splitter**  
now we need to make a function that will split the book into documents.  
For this we have 2 important principles:  
1. Each document contains a complete idea
2. The document is short enough to produce good weights in the assimilation model


In [ ]:
#for the RecursiveCharacterTextSplitter we need to make hierarchy aaray of the splitting characters
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    ".",
    " ",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,#size of documents
    chunk_overlap=80,#overlap of 2 nearby documents
    add_start_index=True,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS,
)

Now , the embbeding function is needed  
The embedding proccess is preformed only once   
the embedded vectors are kept in the local mongoDB 

In [ ]:
# Use a Hebrew BERT model
model_name = "avichr/heBERT"
model = BertModel.from_pretrained(model_name, resume_download=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)


#embedding function
def embeddingVectors(text):
    encoding = tokenizer.batch_encode_plus(
        text,  # List of input texts
        padding=True,  # Pad to the maximum sequence length
        truncation=True,  # Truncate to the maximum sequence length if necessary
        return_tensors='pt',  # Return PyTorch tensors
        add_special_tokens=True  # Add special tokens CLS and SEP
    )
    input_ids = encoding['input_ids']  # Token IDs
    attention_mask = encoding['attention_mask']  # Attention mask

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        word_embeddings = outputs.last_hidden_state  # (batch_size, sequence_length, embedding_dim)
        sentence_embedding = word_embeddings[:, 0, :]  
    return sentence_embedding  # This contains the embeddings


#save in mongo
def save_embedding(word_embeddings,topic):
    client = MongoClient("mongodb://localhost:27017/")
    db = client["halacha_embedding"]
    embedded_texts_collection = db[f"{topic}_embedded_vectors"]
    embedded_texts_collection.insert_many([{"embedding": we.tolist()} for we in word_embeddings])

# embedding and saving proccess    
def process_sentences(sentences,topic):
    word_embeddings = embeddingVectors([doc["text"] for doc in sentences])
    save_embedding(word_embeddings,topic)


# Parent Document Retriever
![Parent Document Retriever](PDR.png)
Parent Document Retriever- the first splliting step  

division of the book according to the division of the Jewish books of Halacha into chapters and sections

In [ ]:
def DataPreprocessing(content, topic):
    current_title = None
    
    current_section = None
    sections = []

    section_letters = ["א", "ב", "ג", "ד", "ה", "ו", "ז", "ח", "ט", "י", "יא", "יב", "יג", "יד", "טו", "טז", "יז", "יח", "יט", "כ", "כא", "כב", "כג", "כד", "כה", "כו", "כז", "כח", "כט", "ל"]

    for line in content:
        line = line.strip()
        line = re.sub(r'\[[^\]]*\]', '', line)
        line = re.sub(r'\([^)]*\)', '', line)
        line = re.sub(r'\.\s\.', '.', line)

        if line.startswith("!"):
            current_title = line[1:].strip() + "."
        elif any(line.startswith(letter) for letter in section_letters):
            if current_section:
                sections.append(current_section)
            current_section = {"title": current_title, "content": line[1:].strip()}
        else:
            if current_section:
                current_section["content"] += " " + line.strip()

    if current_section:
        sections.append(current_section)

    processed_texts = []
    for idx, section in enumerate(sections):
        chunks = text_splitter.split_text(section["content"])
        for chunk in chunks:
            document = {
                "metadata": {"title": section["title"], "subIndex": idx},
                "text": section["title"] + " " + chunk
            }
            processed_texts.append(document)
  
    process_sentences(processed_texts,topic)
    client = MongoClient("mongodb://localhost:27017/")
    db = client["halacha"]
    processed_texts_collection = db[f"{topic}_processed_texts"]
    sections_collection = db[f"{topic}_sections"]

    processed_texts_collection.insert_many(processed_texts)
    sections_collection.insert_many(sections)

loading and saving functions

In [ ]:

def load_data_from_mongodb(topic):
    client = MongoClient("mongodb://localhost:27017/")
    db = client["halacha"]
    processed_texts_collection = db[f"{topic}_processed_texts"]
    sections_collection = db[f"{topic}_sections"]

    processed_texts = list(processed_texts_collection.find())
    sections = list(sections_collection.find())

    return processed_texts, sections



def load_vectors_from_mongodb(topic):
    client = MongoClient("mongodb://localhost:27017/")
    db = client["halacha_embedding"]
    embedded_texts_collection = db[f"{topic}_embedded_vectors"]
    embeddings = list(embedded_texts_collection.find({}, {"_id": 0, "embedding": 1}))
    return [torch.tensor(e["embedding"]) for e in embeddings]



def loadTopics():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["halacha"]
    topics_collection = db["topics"]
    topics = list(topics_collection.find({}))
    return [topic["name"] for topic in topics]


def saveTopic(topic):
    client = MongoClient("mongodb://localhost:27017/")
    db = client["halacha"]
    topics_collection = db["topics"]
    topics_collection.insert_one({"name": topic})

Another function that is needed is a ranking function because the cosine similarity is not accurate enough  
i used a pretrained rerank model  from cohere

In [ ]:
co = cohere.Client('your-cohere-APIkey')
def rerank_documents(query, docs, top_n):
    results = co.rerank(model="rerank-multilingual-v3.0", query=query, documents=docs, top_n=top_n, return_documents=True)
    print("Rerank results:", results)
    reranked_docs = [res.document.text for res in results.results]
    return reranked_docs

Finally - the Rag algorithem
![RAG](rag.png)


In [ ]:
def find_sections_by_title(sections, title):
    return [section["content"] for section in sections if section["title"] == title]


boto3_bedrock = boto3.client(service_name='bedrock-runtime',region_name='us-east-1')


def RAG(question, topic):
    encoded_sentences = load_vectors_from_mongodb(topic)
    encoded_sentences_tensor = torch.stack(encoded_sentences)  # המרת הרשימה לטנסור אחד

    query = [question]
    embedded_query = embeddingVectors(query)

    numpy_embeddings = encoded_sentences_tensor.cpu().numpy()
    index = faiss.IndexFlatL2(encoded_sentences_tensor.shape[1])
    index.add(numpy_embeddings)

    D, I = index.search(embedded_query.cpu().numpy(), k=20)

    processed_texts, sections = load_data_from_mongodb(topic)
    nearest_neighbors_texts = [processed_texts[idx]["text"] for idx in I[0]]

    reranked_texts = rerank_documents(query[0], nearest_neighbors_texts, top_n=20)
    first_rerank=""
    if reranked_texts:
        first_text = reranked_texts[0]
        first_period_index = first_text.find('.')
        if first_period_index != -1 and first_period_index <= 50:
            first_rerank = first_text[first_period_index+1:].strip()
        else:
            first_rerank = first_text    
    processed_texts_2 = []
    for text in reranked_texts:
        first_period_index = text.find('.')
        if first_period_index != -1 and first_period_index <= 50:
            title = text[:first_period_index+1].strip()
            relevant_sections = find_sections_by_title(sections, title)
            combined_content = " ".join(relevant_sections)
            processed_texts_2.append(combined_content)
        else:
            processed_texts_2.append(text)
    top_3_texts = processed_texts_2[:3]
    context = " ".join(top_3_texts)
    client = anthropic.Client(api_key="claude-APIkey")
    response = client.messages.create(
        model="claude-2.1",
        system="Human: אתה רב שעונה על שאלות בהלכה. תסכם בשלוש-ארבע נקודות על פי המידע המצורף שבו יש תשובה לשאלה , אם אתה לא יודע, תכתוב איני יודע.",
        messages=[
            {"role": "user", "content": " Context: " + context + " Question: " + query[0]}
        ],
        max_tokens=400
    )
    answer = ""
    for text_block in response.content:
        answer += text_block.text
    return answer,first_rerank

for Ui I used streamlit

In [ ]:
st.image('logo.png', use_column_width=True)
st.markdown("<h1 style='text-align: center;'>שאל אותי כל שאלה</h1>", unsafe_allow_html=True)

if 'topics' not in st.session_state:
    st.session_state['topics'] =loadTopics()

selected_topic = st.selectbox("בחר נושא", st.session_state['topics'])
question = st.text_input('', placeholder='הכנס את השאלה כאן', key='input', label_visibility='collapsed')
col1, col2, col3 = st.columns([1, 0.5, 1])

with col2:
    submit_button = st.button('שלח')

if submit_button:
    if question:
        placeholder = st.empty()
        placeholder.text("LOADING")

        answer, source  = RAG(question, selected_topic) 

        placeholder.text("")
        st.markdown(f"<div style='direction: rtl; text-align: right;'>מקור: {source}</div> </br>", unsafe_allow_html=True)
        st.markdown(f"<div style='direction: rtl; text-align: right;'>תשובת מענה RAG: {answer}</div>", unsafe_allow_html=True)
    else:
        st.markdown("<div style='direction: rtl; text-align: right;'>אנא הכנס שאלה.</div>", unsafe_allow_html=True)

uploaded_file = st.file_uploader("העלה קובץ טקסט", type=["txt"])

if uploaded_file:
    new_topic = st.text_input("הכנס שם לנושא החדש")

    if new_topic:
        content = uploaded_file.read().decode("utf-8")
        lines = content.split("\n")
        processed_lines = []
        for line in lines:
            if line.startswith("סימן"):
                line = "!" + line
            processed_lines.append(line)

        DataPreprocessing(processed_lines, new_topic)
        saveTopic(new_topic)
        st.success(f"הנושא '{new_topic}' נוסף בהצלחה!")
        st.experimental_rerun()  

